In [1]:
# make rnn model in pytorch

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [31]:
# Create a class for the dataset
class MatchesDetailsDataset(Dataset):
    def __init__(self, matches_info, matches_details, targets):
        self.matches_info = matches_info
        self.matches_details = matches_details
        self.targets = targets

    def __len__(self):
        return len(self.matches_info)

    def __getitem__(self, idx):
        match_info = self.matches_info.iloc[idx].astype(np.float32).values
        match_details = self.matches_details[self.matches_details['id'] == match_info[0]].astype(np.float32).values
        target = self.targets[self.targets['id'] == match_info[0]].astype(np.float32).values

        # remove id
        return match_info[1:], match_details[:, 1:], target[:, 3:]

In [3]:
matches_info = pd.read_csv('matches_info.csv')
matches_details = pd.read_csv('lstm_x_final.csv')
targets = pd.read_csv('lstm_y_final.csv')

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# Create the RNN model
class CricketRNN(nn.Module):
    def __init__(self, input_sequence_features, sequence_length, hidden_size, num_layers, number_of_features, regression=True):
        super(CricketRNN, self).__init__()
        self.hidden_size = hidden_size
        self.regression = regression
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_sequence_features, hidden_size, num_layers, nonlinearity='relu' ,batch_first=True)
        self.output_fc = nn.Sequential(
            nn.Linear(hidden_size * sequence_length, sequence_length),
        )
        self.h0_fc = nn.Sequential(
            nn.Linear(number_of_features, self.num_layers * self.hidden_size),
            nn.ReLU()
        )

    def forward(self, x, s, h0 = None):
        if h0 is None:
            h0 = torch.reshape(self.h0_fc(x), (self.num_layers, x.size(0), self.hidden_size)).requires_grad_()

        out, hn = self.rnn(s, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.output_fc(out)
        out = torch.relu(out) if self.regression else torch.tanh(out)
        return out, hn

In [37]:
# train the model
def train_model(model_runs, model_winner, model_final, train_loader, test_loader, optimizer_runs, optimizer_winner, optimizer_final, criterion, epochs):
    for epoch in range(epochs):
        model_runs.train()
        model_winner.train()
        model_final.train()

        for x, s, y in train_loader:
            optimizer_runs.zero_grad()

            x = x.to(device)
            s = s.to(device)
            y = y.to(device)

            # Forward pass
            out_runs, hn_runs = model_runs(x, s)
            train_loss_runs = criterion(out_runs, y[:, :, 0])

            # Backward and optimize
            train_loss_runs.backward()
            optimizer_runs.step()

            optimizer_winner.zero_grad()

            # Forward pass
            out_winner, hn_winner = model_winner(x, s)
            train_loss_winner = criterion(out_winner, y[:, :, 1])

            # Backward and optimize
            train_loss_winner.backward()
            optimizer_winner.step()

            optimizer_final.zero_grad()

            # Forward pass
            out_final, hn_final = model_final(x, s)
            train_loss_final = criterion(out_final, y[:, :, 2])

            # Backward and optimize
            train_loss_final.backward()
            optimizer_final.step()

        
        # evaluate model
        model_runs.eval()
        model_winner.eval()
        model_final.eval()
        with torch.no_grad():
            for x, s, y in test_loader:
                x = x.to(device)
                s = s.to(device)
                y = y.to(device)

                # Forward pass
                out_runs, _ = model_runs(x, s)
                test_loss_runs = criterion(out_runs, y[:, :, 0])

                out_winner, _ = model_winner(x, s)
                test_loss_winner = criterion(out_winner, y[:, :, 1])

                out_final, _ = model_final(x, s)
                test_loss_final = criterion(out_final, y[:, :, 2])

        print(f'Epoch [{epoch+1}/{epochs}], Train Runs: {train_loss_runs.item():.4f}, Test Runs: {test_loss_runs.item():.4f} Train Winner: {train_loss_winner.item():.4f}, Test Winner: {test_loss_winner.item():.4f} Train Final: {train_loss_final.item():.4f}, Test Final: {test_loss_final.item():.4f}')

In [27]:
input_sequence_features = 19 # Hin
sequence_length = 40 # L
hidden_size = 40 # Hout
num_layers = 3
number_of_features = 30

In [32]:
# model setup
model_runs = CricketRNN(input_sequence_features, sequence_length, hidden_size, num_layers, number_of_features)
model_winner = CricketRNN(input_sequence_features, sequence_length, hidden_size, num_layers, number_of_features, regression=False)
model_final = CricketRNN(input_sequence_features, sequence_length, hidden_size, num_layers, number_of_features)


criterion = nn.MSELoss()
    
# Optimizer with l2 regularization
optimizer_runs = optim.Adam(model_runs.parameters(), lr=0.001, weight_decay=0.0001)
optimizer_winner = optim.Adam(model_winner.parameters(), lr=0.001, weight_decay=0.0001)
optimizer_final = optim.Adam(model_final.parameters(), lr=0.001, weight_decay=0.0001)

model_runs.to(device)
model_winner.to(device)
model_final.to(device)

CricketRNN(
  (rnn): RNN(19, 40, num_layers=3, batch_first=True)
  (output_fc): Sequential(
    (0): Linear(in_features=1600, out_features=40, bias=True)
  )
  (h0_fc): Sequential(
    (0): Linear(in_features=30, out_features=120, bias=True)
    (1): ReLU()
  )
)

In [33]:
train, test = train_test_split(matches_info, test_size=0.2, random_state=42)

In [34]:
train_loader = DataLoader(MatchesDetailsDataset(train, matches_details, targets), batch_size=32, shuffle=False)

In [35]:
test_loader = DataLoader(MatchesDetailsDataset(test, matches_details, targets), batch_size=32, shuffle=False)

In [38]:
# train the model
train_model(model_runs, model_winner, model_final, train_loader, test_loader, optimizer_runs, optimizer_winner, optimizer_final, criterion, 200)

Epoch [1/200], Train Runs: 549.8566, Test Runs: 713.4745 Train Winner: 0.3450, Test Winner: 0.6683 Train Final: 11456.4746, Test Final: 14295.4648
Epoch [2/200], Train Runs: 549.2979, Test Runs: 713.2053 Train Winner: 0.4738, Test Winner: 0.6525 Train Final: 11416.7178, Test Final: 14243.1875
Epoch [3/200], Train Runs: 549.0011, Test Runs: 712.7888 Train Winner: 0.3843, Test Winner: 0.6631 Train Final: 11404.3906, Test Final: 14179.1084
Epoch [4/200], Train Runs: 548.7251, Test Runs: 712.4807 Train Winner: 0.2629, Test Winner: 0.6856 Train Final: 11392.6211, Test Final: 14139.8545
Epoch [5/200], Train Runs: 119.4871, Test Runs: 127.5081 Train Winner: 0.2302, Test Winner: 0.6870 Train Final: 11387.1787, Test Final: 14129.4072
Epoch [6/200], Train Runs: 111.8599, Test Runs: 122.2161 Train Winner: 0.1744, Test Winner: 0.6872 Train Final: 11381.9824, Test Final: 14125.0625
Epoch [7/200], Train Runs: 111.0636, Test Runs: 121.6634 Train Winner: 0.1389, Test Winner: 0.7128 Train Final: 11376.

KeyboardInterrupt: 